## Creating training data for cellpose segmentation

From the ROI images, extract a set of representative image slices

In [17]:
%%capture
#suppresses output
import numpy as np #importing some packages we will need
import skimage.io
import matplotlib.pyplot as plt
import sys
sys.path.append('/home/groups/ZuckermanLab/copperma/cell/celltraj/celltraj')
import imageprep as imprep
import utilities
import skimage.filters
import scipy.ndimage
from skimage.filters import threshold_local
from skimage.segmentation import clear_border
import cmapfile
import pathlib
import nd2
from skimage.measure import regionprops_table
from IPython.display import clear_output
import os

##### Establish channels, imaging parameters, and object parameters-- use NIS AR or other software first to gather info, copy from extract_roi step

In [20]:
date = '2024.03.08'
modelName = f'20240302_OS17lung_RM' #results will be saved/named using this string
nxy=20
ind_xy = np.arange(nxy).astype(int)
#ind_xy = np.array([3,4]).astype(int) #choose xys for analysis. If all, use ind_xy = np.arange(nxy).astype(int)
nuc_channel=0 #nuclear channel
rep_channel=3 #reporter channel (ERK here)
bf_channel=4 #bright-field channel
channels = ['DAPI','CFP','TRITC','YFP','BF'] #names for channels in plots
channel_colors = ['blue','cyan','red','goldenrod','gray'] #colors for channels in plots
channel_dict=dict([(x, i) for i, x in enumerate(channels)])
micron_per_pixel=.33 #lateral resolution (image pixel size)
micron_per_z=3.0 #axial resolution between z-slices
zscale=micron_per_z/micron_per_pixel
nucl=40 #typical nucleus linear diameter pixels

In [21]:
#some convenience functions for later
def show_image_3channel(bf_img,nuc_img,rep_img,ax=None):
        if ax is None:
            ax=plt.gca()
        ax.imshow(imprep.znorm(bf_img),cmap=plt.cm.binary,clim=(-5,5))
        cs=ax.contour(imprep.znorm(nuc_img),cmap=plt.cm.BuPu,levels=np.linspace(0,np.percentile(imprep.znorm(nuc_img),99),7),alpha=.5,linewidths=.5)
        cs.cmap.set_over('purple')
        cs=ax.contour(imprep.znorm(rep_img),cmap=plt.cm.YlOrBr_r,levels=np.linspace(0,np.percentile(imprep.znorm(rep_img),99),7),alpha=.25,linewidths=.5)
        cs.cmap.set_over('yellow')
        return ax

def rescale_to_int(img,maxint=2**16-1,dtype=np.uint16):
    img=maxint*((img-np.min(img))/np.max(img-np.min(img)))
    return img.astype(dtype)

#### Training set creation

We are going to extract a set of representative image slices from each ROI and save in a folder for cellpose training

In [15]:
visual=False #try with visual=True first just to see if slices look good
nz_random=1
nlateral_random=0
nt_train=3
itrain=0
datapath='/home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM'
trainpath='/home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata'
int_cut=1.2
highlow_perc=.1
int_scale=.1 #scaling between brightfield in fluorescence
nz_contiguous=2

In [11]:
pathexists = os.path.isdir(trainpath) #create training folder if it doesn't exist already
if not pathexists:
    os.mkdir(trainpath)

Now loop through ROIs and extract some slices

In [16]:
for ixy in ind_xy:
    imagelist = imprep.list_images(f'{datapath}/{modelName}_XY{ixy}_roi*.tif') #get list of all ROI images per XY
    n_rois=len(imagelist)
    for i_roi in range(n_rois):
        imgs_roi=skimage.io.imread(imagelist[i_roi])
        nt_roi,nz_roi,nx_roi,ny_roi,nchannels_roi=imgs_roi.shape
        train_frames=np.linspace(0,nt_roi-1,nt_train).astype(int)
        for itt in range(0,nt_train):
            iS=train_frames[itt]
            img=imgs_roi[iS,...].astype(float)
            int_low=np.percentile(img,highlow_perc,axis=(0,1,2))
            int_high=np.percentile(img,100.-highlow_perc,axis=(0,1,2))
            int_ratio=np.divide(int_high,int_low) #estimate of the intensity variation in the channel
            im3d_nuc=rescale_to_int(imprep.znorm(img[...,nuc_channel]))
            im3d_rep=rescale_to_int(imprep.znorm(img[...,rep_channel]))
            im3d_bf=imprep.znorm(img[...,bf_channel])
            im3d_bf=skimage.filters.difference_of_gaussians(im3d_bf,.5,1.5)
            im3d_bf=rescale_to_int(im3d_bf)
            cellpose_img=np.zeros((nz_roi,3,nx_roi,ny_roi)).astype(np.uint16)
            cellpose_img[:,0,:,:]=im3d_bf #.5*(im3d_rep+im3d_nuc) #add in nuc so not hollow
            cellpose_img[:,1,:,:]=im3d_rep
            cellpose_img[:,2,:,:]=im3d_nuc
            try:
                if (int_ratio<int_cut).any(): #extract slices from rois with meaningful intensity variation
                    raise Exception(f"Intensity high/low percentile ratios {int_ratio[0]:.2f},{int_ratio[1]:.2f},{int_ratio[2]:.2f} less than specified cutoff {int_cut}: xy{ixy} roi{i_roi} t{iS}")
                    if visual:
                        clear_output(wait=True);plt.clf();plt.imshow(np.max(cellpose_img[:,0,:,:],axis=0));plt.pause(.5);
                indsz=np.argsort(np.mean(im3d_nuc,axis=(1,2)))[-2*nz_random:] #np.where(np.mean(im3d_nuc,axis=(1,2))>np.mean(im3d_nuc)+.1*np.std(im3d_nuc))[0]
                z_inds=np.random.choice(indsz,size=nz_random,replace=False).astype(int)
                indsx=np.argsort(np.mean(im3d_nuc,axis=(0,2)))[-2*nlateral_random:] #np.where(np.mean(im3d_nuc,axis=(0,2))>np.mean(im3d_nuc)+.1*np.std(im3d_nuc))[0]
                x_inds=np.random.choice(indsx,size=nlateral_random,replace=False).astype(int)
                indsy=np.argsort(np.mean(im3d_nuc,axis=(0,1)))[-2*nz_random:] #np.where(np.mean(im3d_nuc,axis=(0,1))>np.mean(im3d_nuc)+.1.*np.std(im3d_nuc))[0]
                y_inds=np.random.choice(indsy,size=nlateral_random,replace=False).astype(int)
                ix=np.argsort(np.sum(np.sum(im3d_nuc,axis=2),axis=0))[-1]
                iy=np.argsort(np.sum(np.sum(im3d_nuc,axis=1),axis=0))[-1]
                imgsz=[None]*nz_random
                for iz in range(nz_random):
                    zrange=np.arange(z_inds[iz]-nz_contiguous,z_inds[iz]+nz_contiguous+1)
                    zrange=zrange[zrange>=0];zrange=zrange[zrange<nz_roi]
                    imgsz[iz]=cellpose_img[z_inds[iz],:,:,:]
                    for izc in zrange:
                        imgz=cellpose_img[izc,:,:,:]
                        fname=f'{trainpath}/train{itrain}_t{iS+1}_z{z_inds[iz]+1}_xy{ixy+1}_roi{i_roi}.tif'
                        skimage.io.imsave(fname,imgz.T)
                        itrain=itrain+1
                for ilat in range(nlateral_random):
                    ix=x_inds[ilat]
                    iy=y_inds[ilat]
                    imgy=cellpose_img[:,:,:,iy]
                    imgy=skimage.transform.rescale(imgy,(zscale,1,1),anti_aliasing=False)
                    imgy=np.swapaxes(imgy,1,0)
                    fname=f'{trainpath}/train{itrain}_t{iS+1}_y{iy+1}_xy{ixy+1}_roi{i_roi}.tif'
                    skimage.io.imsave(fname,imgy.T)
                    itrain=itrain+1
                    imgx=cellpose_img[:,:,ix,:]
                    imgx=skimage.transform.rescale(imgx,(zscale,1,1),anti_aliasing=False)
                    imgx=np.swapaxes(imgx,1,0)
                    fname=f'{trainpath}/train{itrain}_t{iS+1}_x{ix+1}_xy{ixy+1}_roi{i_roi}.tif'
                    skimage.io.imsave(fname,imgx.T)
                    itrain=itrain+1
                    if visual:
                        clear_output(wait=True);plt.clf()
                        for iz in range(nz_random):
                            plt.subplot(1,3,1)
                            show_image_3channel(imgsz[iz][2,...],imgsz[iz][1,...],imgsz[iz][0,...]);plt.axis('off')
                            plt.subplot(1,3,2)
                            show_image_3channel(imgx[2,...],imgx[1,...],imgx[0,...]);plt.axis('off')
                            plt.subplot(1,3,3)
                            show_image_3channel(imgy[2,...],imgy[1,...],imgy[0,...]);plt.axis('off')
                            plt.pause(.1)
            except Exception as e:
                print(e)

/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train0_t1_z5_xy1_roi0.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)
/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train1_t1_z5_xy1_roi0.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)
/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train2_t1_z5_xy1_roi0.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)


Intensity high/low percentile ratios 1.87,1.18,1.25 less than specified cutoff 1.2: xy0 roi0 t36
Intensity high/low percentile ratios 1.89,1.17,1.24 less than specified cutoff 1.2: xy0 roi0 t72
Intensity high/low percentile ratios 1.72,1.18,1.29 less than specified cutoff 1.2: xy1 roi0 t36
Intensity high/low percentile ratios 1.81,1.18,1.28 less than specified cutoff 1.2: xy1 roi0 t72
Intensity high/low percentile ratios 2.23,1.18,1.25 less than specified cutoff 1.2: xy1 roi1 t36
Intensity high/low percentile ratios 2.50,1.18,1.23 less than specified cutoff 1.2: xy1 roi1 t72


/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train17_t1_z3_xy3_roi0.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)


Intensity high/low percentile ratios 1.80,1.18,1.30 less than specified cutoff 1.2: xy2 roi0 t36
Intensity high/low percentile ratios 1.82,1.18,1.28 less than specified cutoff 1.2: xy2 roi0 t72
Intensity high/low percentile ratios 2.37,1.20,1.32 less than specified cutoff 1.2: xy2 roi1 t36
Intensity high/low percentile ratios 2.58,1.19,1.29 less than specified cutoff 1.2: xy2 roi1 t72
Intensity high/low percentile ratios 2.10,1.17,1.23 less than specified cutoff 1.2: xy2 roi2 t36
Intensity high/low percentile ratios 2.16,1.17,1.22 less than specified cutoff 1.2: xy2 roi2 t72


/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train28_t1_z3_xy4_roi0.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)
/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train29_t1_z3_xy4_roi0.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)
/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train30_t1_z3_xy4_roi0.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)
/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train31_t1_z3_xy4_roi0.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)
/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/m

...

Intensity high/low percentile ratios 2.75,1.19,1.26 less than specified cutoff 1.2: xy17 roi0 t36
Intensity high/low percentile ratios 2.85,1.18,1.24 less than specified cutoff 1.2: xy17 roi0 t72
Intensity high/low percentile ratios 2.68,1.20,1.25 less than specified cutoff 1.2: xy17 roi1 t72
Intensity high/low percentile ratios 2.16,1.18,1.25 less than specified cutoff 1.2: xy18 roi0 t36
Intensity high/low percentile ratios 2.17,1.18,1.23 less than specified cutoff 1.2: xy18 roi0 t72


/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train248_t37_z1_xy19_roi1.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)
/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train249_t37_z1_xy19_roi1.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)
/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train250_t37_z1_xy19_roi1.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)


Intensity high/low percentile ratios 2.28,1.19,1.29 less than specified cutoff 1.2: xy18 roi1 t72
Intensity high/low percentile ratios 2.53,1.19,1.22 less than specified cutoff 1.2: xy19 roi0 t36
Intensity high/low percentile ratios 2.61,1.18,1.21 less than specified cutoff 1.2: xy19 roi0 t72


/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train254_t1_z8_xy20_roi1.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)
/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train255_t1_z8_xy20_roi1.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)
/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train256_t1_z8_xy20_roi1.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)
/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-cell-sys/makkawi/20240302_OS17lung_RM/cellpose_trainingdata/train257_t1_z8_xy20_roi1.tif is a low contrast image
  skimage.io.imsave(fname,imgz.T)
/tmp/ipykernel_3516/238470769.py:44: UserWarning: /home/groups/CEDAR/copperma/dyn-ce

Intensity high/low percentile ratios 3.27,1.20,1.25 less than specified cutoff 1.2: xy19 roi1 t36
Intensity high/low percentile ratios 3.26,1.19,1.23 less than specified cutoff 1.2: xy19 roi1 t72
Intensity high/low percentile ratios 2.22,1.20,1.24 less than specified cutoff 1.2: xy19 roi2 t0
Intensity high/low percentile ratios 2.24,1.18,1.22 less than specified cutoff 1.2: xy19 roi2 t36
Intensity high/low percentile ratios 2.24,1.18,1.20 less than specified cutoff 1.2: xy19 roi2 t72


In [10]:
cellpose_img

array([[[[28123, 25132, 22899, ..., 29370, 25446, 30083],
         [ 1419, 26806, 31505, ..., 29420, 21355, 25731],
         [32290, 25681, 26328, ...,  2116, 28359, 25371],
         ...,
         [24637, 27628, 25068, ...,   202, 29592, 23050],
         [23013, 18593, 28753, ..., 31216, 24293, 23784],
         [21393, 27096,  1772, ..., 20234, 31455, 27951]],

        [[30136, 20569, 25831, ..., 31093, 33485, 25352],
         [39703, 23917, 35876, ..., 29658, 11480, 25352],
         [35398, 26788, 22961, ..., 34441, 23439, 21047],
         ...,
         [20091, 29658, 29658, ..., 30614, 32049, 21526],
         [15307, 17220, 26788, ..., 36833, 15307, 22482],
         [17699, 22961, 29658, ..., 21526, 37311, 27744]],

        [[14069, 13189, 11824, ..., 12393, 10109, 14237],
         [11832, 13278, 12066, ..., 13153,  9603, 12353],
         [13591, 12881, 12414, ..., 13152, 12342, 13550],
         ...,
         [15803, 15548, 13557, ..., 12716, 12852, 12273],
         [13016, 12627, 11